In [2]:
# BE CAREFUL, this will cause a version change in your system, use in a virtual environment
%pip install networkx==3.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
import networkx as nx
import pandas as pd
import community.community_louvain as community #python-louvain

In [7]:
#read network tables
human_GM = pd.read_table('../networks/human_netwoks.txt', header=None)
human = pd.read_table('../networks/human_another_netwoks.txt', header=None)
yeast_GM = pd.read_table('../networks/yeast_gm_netwoks.txt', header=None)
yeast = pd.read_table('../networks/yeast_netwoks.txt', header=None)
yeast_GM_filter = pd.read_table('../networks/yeast_filter_networks.txt', header=None)
networks = [yeast, yeast_GM, yeast_GM_filter, human, human_GM]

In [8]:
#create graphs 
graphs = []
for network in networks:
    g = network.drop(network.columns[[2]], axis=1)
    g.columns = ['A', 'B']
    
    graphs.append(nx.from_pandas_edgelist(g, source='A', target='B'))


In [9]:
#modularity calculation
part = [community.best_partition(g) for g in graphs]
mod = [community.modularity(part[i], graphs[i]) for i in range(len(graphs))]

In [10]:
df = pd.DataFrame(mod, index =['yeast', 'yeast_GM', 'yeast_GM_filter', 'human', 'human_GM'],
                                              columns =['modularity'])
df

,modularity
yeast,0.863634
yeast_GM,0.533725
yeast_GM_filter,0.914655
human,0.727528
human_GM,0.588689


## Annotation for partitions

In [11]:
domains = []

for net in part:
    new_dict = {}
    for key, value in net.items():
        if value in new_dict:
            new_dict[value].append(key)
        else:
            new_dict[value]=[key]
    domains.append(new_dict)
        

In [13]:
human_domains = pd.DataFrame({'domain' : list(domains[3].keys()), 
                              'count' : [len(lst) for lst in domains[3].values()], 
                              'genes' : list(domains[3].values())})
human_domains.sort_values(by=['count'], ascending=False).to_csv('data/human_domains.csv')